<a href="https://colab.research.google.com/github/dFarrulla/emd-desafio-junior-data-scientist/blob/main/analise_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Instalação da Biblioteca `basedosdados`

In [1]:
pip install basedosdados


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.1/43.1 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.5/97.5 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.0/204.0 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.2/135.2 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.0/106.0 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.6/133.6 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.6/25.6 MB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.4/108.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.9/70.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [1]:
import basedosdados as bd

###1. Quantos chamados foram abertos no dia 01/04/2023?

In [2]:
# Definimos a query
query1 = """
SELECT
    COUNT(*) AS chamados
FROM
    `datario.adm_central_atendimento_1746.chamado`
WHERE
    DATE(data_inicio) = '2023-04-01'
"""

In [3]:
# o ID do projeto a ser 'cobrado', de acordo com recomendação do github da lib
bd.config.billing_project_id = 'projetorio-431817'

In [4]:
# criando o Pandas DataFrame com a query
df_query1 = bd.read_sql(query1)

Downloading: 100%|██████████| 1/1 [00:00<00:00,  4.70rows/s]


In [5]:
# Visualizando a tabela
df_query1.head()

,chamados
0,1756


###R: 1756 chamados foram abertoes neste dia.



###2. Qual o tipo de chamado que teve mais teve chamados abertos no dia 01/04/2023?

In [6]:
# Definimos a query
query2 = """
SELECT
    tipo, COUNT(*) AS chamadas_por_tipo
FROM
    `datario.adm_central_atendimento_1746.chamado`
WHERE
    DATE(data_inicio) = '2023-04-01'
GROUP BY
    tipo
ORDER BY
    chamadas_por_tipo DESC
"""

# criando o Pandas DataFrame
df_query2 = bd.read_sql(query2, billing_project_id='projetorio-431817')
df_query2.head()

Downloading: 100%|██████████| 55/55 [00:00<00:00, 245.88rows/s]


,tipo,chamadas_por_tipo
0,Estacionamento irregular,366
1,Remoção Gratuita,196
2,Manutenção de iluminação pública,122
3,Limpeza de logradouros,116
4,Perturbação do sossego,111


###R: Estacionamento irregular com 366 chamados abertos.

###3. Quais os nomes dos 3 bairros que mais tiveram chamados abertos nesse dia?

In [7]:
query3 = """
SELECT
  `datario.adm_central_atendimento_1746.chamado`.id_bairro,
  `datario.dados_mestres.bairro`.nome as nome_bairro,
  COUNT(*) AS qtd_chamado
FROM
  `datario.adm_central_atendimento_1746.chamado`
LEFT JOIN
  `datario.dados_mestres.bairro`
  ON `datario.adm_central_atendimento_1746.chamado`.id_bairro = `datario.dados_mestres.bairro`.id_bairro
WHERE
  DATE(data_inicio) = '2023-04-01'
GROUP BY
  id_bairro, nome_bairro
ORDER BY
  qtd_chamado DESC
"""

# criando o Pandas DataFrame
df_query3 = bd.read_sql(query3, billing_project_id='projetorio-431817')

df_query3.head()

Downloading: 100%|██████████| 146/146 [00:00<00:00, 621.16rows/s]


,id_bairro,nome_bairro,qtd_chamado
0,144,Campo Grande,113
1,33,Tijuca,89
2,None,None,73
3,128,Barra da Tijuca,59
4,141,Bangu,53


###R: Campo Grande, Tijuca e Barra da Tijuca. Vale destacar que existem 73 chamados que não estão vinculados a nenhum bairro específico, o que resultou na inclusão do bairro Barra da Tijuca entre os três primeiros.

###4. Qual o nome da subprefeitura com mais chamados abertos nesse dia?


In [8]:
query4 = """
SELECT
  `datario.dados_mestres.bairro`.subprefeitura,
  COUNT(*) AS qtd_chamado
FROM
  `datario.adm_central_atendimento_1746.chamado`
LEFT JOIN
  `datario.dados_mestres.bairro`
  ON `datario.adm_central_atendimento_1746.chamado`.id_bairro = `datario.dados_mestres.bairro`.id_bairro
WHERE
  DATE(data_inicio) = '2023-04-01'
GROUP BY
  subprefeitura
ORDER BY
  qtd_chamado DESC
"""

# criando o Pandas DataFrame
df_query4 = bd.read_sql(query4, billing_project_id="projetorio-431817")
df_query4.head()

Downloading: 100%|██████████| 10/10 [00:00<00:00, 45.92rows/s]


,subprefeitura,qtd_chamado
0,Zona Norte,510
1,Zona Oeste,238
2,Zona Sul,233
3,Grande Tijuca,159
4,Jacarepaguá,145


###R: subprefeitura 'Zona Norte', com 510 chamados abertos.

###5. Existe algum chamado aberto nesse dia que não foi associado a um bairro ou subprefeitura na tabela de bairros? Se sim, por que isso acontece?5. Existe algum chamado aberto nesse dia que não foi associado a um bairro ou subprefeitura na tabela de bairros? Se sim, por que isso acontece?

In [9]:
query5 = """
SELECT
  `datario.dados_mestres.bairro`.nome as nome_bairro,
  `datario.dados_mestres.bairro`.subprefeitura,
  COUNT(*) AS qtd_chamado
FROM
  `datario.adm_central_atendimento_1746.chamado`
LEFT JOIN
  `datario.dados_mestres.bairro`
  ON `datario.adm_central_atendimento_1746.chamado`.id_bairro = `datario.dados_mestres.bairro`.id_bairro
WHERE
  DATE(data_inicio) = '2023-04-01'
GROUP BY
  nome_bairro, subprefeitura
ORDER BY
  qtd_chamado DESC
"""

# criando o Pandas DataFrame
df_query5 = bd.read_sql(query5, billing_project_id="projetorio-431817")
df_query5.head()

Downloading: 100%|██████████| 146/146 [00:00<00:00, 612.15rows/s]


,nome_bairro,subprefeitura,qtd_chamado
0,Campo Grande,Zona Oeste,113
1,Tijuca,Grande Tijuca,89
2,None,None,73
3,Barra da Tijuca,Barra da Tijuca,59
4,Bangu,Grande Bangu,53


###R: Sim, há 73 chamados abertos que não foram vinculados a um bairro ou subprefeitura. Isso pode ter ocorrido por dois motivos:

###Erro de digitação ou inserção de dados; ou
A informação de bairro/subprefeitura foi corrompida pelo código do sistema responsável pela inserção.

###6. Quantos chamados com o subtipo "Perturbação do sossego" foram abertos desde 01/01/2022 até 31/12/2023 (incluindo extremidades)?

In [10]:
query6 = """
SELECT
  subtipo, COUNT(*) as chamados_por_subtipo
FROM
  `datario.adm_central_atendimento_1746.chamado`
WHERE
  DATE(data_inicio) BETWEEN '2022-01-01' AND '2023-12-31' AND subtipo = 'Perturbação do sossego'
GROUP BY
  subtipo
"""

# criando o Pandas DataFrame
df_query6 = bd.read_sql(query6, billing_project_id="projetorio-431817")
df_query6.head()

Downloading: 100%|██████████| 1/1 [00:00<00:00,  4.53rows/s]


,subtipo,chamados_por_subtipo
0,Perturbação do sossego,42830


###R: 42.830 chamados abertos no período acima mencionado, incluindo extremidades.

###7. Selecione os chamados com esse subtipo que foram abertos durante os eventos contidos na tabela de eventos (Reveillon, Carnaval e Rock in Rio).

In [15]:
query7 = """
SELECT
  *
FROM
  `datario.adm_central_atendimento_1746.chamado`
RIGHT JOIN `datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos`
  ON DATE(`datario.adm_central_atendimento_1746.chamado`.data_inicio)
    BETWEEN
      DATE(`datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos`.data_inicial) AND DATE(`datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos`.data_final)
WHERE
  subtipo = 'Perturbação do sossego'
"""

# criando o Pandas DataFrame
df_query7 = bd.read_sql(query7, billing_project_id="projetorio-431817")

df_query7.head()

df_query7.shape

Downloading: 100%|██████████| 1214/1214 [00:00<00:00, 1460.49rows/s]


(1214, 37)

### R: Selecionados.

###8. Quantos chamados desse subtipo foram abertos em cada evento?

In [ ]:
query8 = """
SELECT
  evento,
  COUNT(*) as chamados_por_evento
FROM
  `datario.adm_central_atendimento_1746.chamado`
RIGHT JOIN `datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos`
  ON DATE(`datario.adm_central_atendimento_1746.chamado`.data_inicio)
    BETWEEN
      DATE(`datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos`.data_inicial) AND DATE(`datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos`.data_final)
WHERE
  subtipo = 'Perturbação do sossego'
GROUP BY
  evento
"""

# criando o Pandas DataFrame
df_query8 = bd.read_sql(query8, billing_project_id="projetorio-431817")
df_query8.head()

Downloading: 100%|██████████| 3/3 [00:00<00:00, 23.15rows/s]


,evento,chamados_por_evento
0,Rock in Rio,1668
1,Reveillon,278
2,Carnaval,482


R: 834 chamados durante os dois períodos do Rock in Rio, 241 chamados durante o Carnaval e 139 chamados durante o Reveillon.

###9. Qual evento teve a maior média diária de chamados abertos desse subtipo?
Para isso, criarei uma CTE e utilizarei a média da contagem por dia de evento.

In [ ]:
query9 = """
WITH diarias AS (
  SELECT
    evento,
    DATE(data_inicio) AS data_chamado,
    COUNT(*) AS chamados_diarios
  FROM
    `datario.adm_central_atendimento_1746.chamado`
  RIGHT JOIN `datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos`
    ON DATE(`datario.adm_central_atendimento_1746.chamado`.data_inicio) BETWEEN DATE(`datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos`.data_inicial) AND DATE(`datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos`.data_final)
  WHERE
    subtipo = 'Perturbação do sossego'
  GROUP BY
    evento,
    DATE(data_inicio)
)
SELECT
  evento,
  AVG(chamados_diarios) AS media_chamados_por_evento
FROM
  diarias
GROUP BY
  evento"""

# criando o Pandas DataFrame
df_query9 = bd.read_sql(query9, billing_project_id="projetorio-431817")
df_query9.head()

Downloading: 100%|██████████| 3/3 [00:00<00:00, 20.26rows/s]


,evento,media_chamados_por_evento
0,Carnaval,120.500000
1,Reveillon,92.666667
2,Rock in Rio,238.285714


R: Rock in Rio juntando os dois períodos com 119 chamados abertos por dia de evento.

10. Compare as médias diárias de chamados abertos desse subtipo durante os eventos específicos (Reveillon, Carnaval e Rock in Rio) e a média diária de chamados abertos desse subtipo considerando todo o período de 01/01/2022 até 31/12/2023.

In [ ]:
query10 = """
WITH diarias AS (
  SELECT
    evento,
    DATE(data_inicio) AS data_chamado,
    COUNT(*) AS chamados_diarios
  FROM
    `datario.adm_central_atendimento_1746.chamado`
  RIGHT JOIN `datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos`
    ON DATE(`datario.adm_central_atendimento_1746.chamado`.data_inicio) BETWEEN DATE(`datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos`.data_inicial) AND DATE(`datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos`.data_final)
  WHERE
    subtipo = 'Perturbação do sossego'
  GROUP BY
    evento,
    DATE(data_inicio)
),
media_chamados_por_evento AS (
SELECT
  evento,
  AVG(chamados_diarios) AS media_chamados_por_evento
FROM
  diarias
GROUP BY
  evento
),
chamados_diarios_ano AS (
  SELECT
    DATE(data_inicio) AS data_chamada,
    COUNT(*) AS chamados_diarios
  FROM
    `datario.adm_central_atendimento_1746.chamado`
  WHERE
    DATE(data_inicio) BETWEEN '2022-01-01' AND '2023-12-31'
    AND subtipo = 'Perturbação do sossego'
  GROUP BY
    DATE(data_inicio)
),
media_chamados_ano AS (
SELECT
  AVG(chamados_diarios) AS media_diaria_chamados
FROM
  chamados_diarios_ano
)
SELECT
  m.evento,
  m.media_chamados_por_evento,
  d.media_diaria_chamados
FROM
  media_chamados_por_evento AS m
CROSS JOIN
  media_chamados_ano AS d
  """

# criando o Pandas DataFrame
df_query10 = bd.read_sql(query10, billing_project_id="projetorio-431817")
df_query10.head()

Downloading: 100%|██████████| 3/3 [00:00<00:00, 19.77rows/s]


,evento,media_chamados_por_evento,media_diaria_chamados
0,Rock in Rio,238.285714,61.982634
1,Carnaval,120.500000,61.982634
2,Reveillon,92.666667,61.982634


###R: Criei CTEs com as queries das questões anteriores e utilizei um CROSS JOIN para comparar os períodos, conforme descrito acima. A proporção de chamados abertos no período mencionado em comparação com o período dos eventos é semelhante, exceto no caso do evento Rock in Rio, que apresenta quase o dobro de chamados.